In [13]:
import json
from tqdm.auto import tqdm
import numpy as np
import torch
from transformers import BertTokenizer, BertModel

# 1. Carregando o modelo BERT
model = BertModel.from_pretrained('bert-base-uncased').to('cuda')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 2. Carregando o dataset
data = []
with open(r"C:\Users\heloi\Downloads\yelp_dataset\yelp_academic_dataset_tip.json", encoding="utf-8") as f:
    for line in tqdm(f):
        review = json.loads(line)
        data.append(review['text'])

# 3. Criando o embedding
embeddings = []
for text in tqdm(data):
    # Tokenizando o texto e adicionando os tokens [CLS] e [SEP]
    tokens = tokenizer.encode(text, add_special_tokens=True, max_length=512)

    # Dividindo os tokens em segmentos menores
    MAX_SEGMENT_SIZE = 256
    segmented_tokens = [tokens[i:i+MAX_SEGMENT_SIZE] for i in range(0, len(tokens), MAX_SEGMENT_SIZE)]

    # Obtendo o embedding para cada segmento
    segment_embeddings = []
    for segment in segmented_tokens:
        input_ids = torch.tensor(segment).unsqueeze(0).to('cuda')
        with torch.no_grad():
            output = model(input_ids)
        segment_embeddings.append(output[0][:, 0, :].cpu().numpy())

    # Combinando os embeddings dos segmentos em um único vetor
    embeddings.append(np.concatenate(segment_embeddings, axis=0))

embeddings = np.concatenate(embeddings, axis=0)


# 4. Salvando os embeddings
np.save('embeddingstextreview.npy', embeddings)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0it [00:00, ?it/s]

  0%|          | 0/908915 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [14]:
import numpy as np

# Carrega o arquivo de embeddings
embeddings = np.load('embeddingstextreview.npy')

# Verifica as dimensões dos embeddings
print(embeddings.shape)

(908915, 768)
